## Настройки Colab

In [ ]:
# Настройка пользователя (сделать один раз)
!git config --global user.email "nabludatellip@gmail.com"
!git config --global user.name "ProninPV"

In [ ]:
!git clone https://github.com/ProninPV/ml-regression_concrete-strength.git
%cd ml-regression_concrete-strength

In [ ]:
!pip install catboost

## 6.0 Импорты библиотек

In [4]:
import os
import yaml
import logging
import pickle
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene
from scipy.stats import ttest_ind
from typing import List, Any, Optional, Tuple, Dict, Union
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, KFold, RepeatedKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import time
import psutil
from tqdm import tqdm
import gc

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [8]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [9]:
# В Colab проект клонируется в /content/
# Устанавливаем правильную рабочую директорию
# project_root = Path('/content/ml-regression_concrete-strength')

# Определяем корень проекта
cwd = Path().resolve()
project_root = cwd.parent

# Добавляем корень проекта в sys.path (этого достаточно)
sys.path.append(str(project_root))

# Проверяем наличие конфиг файла
config_path = project_root / "config" / "config.yaml"
print(f"Looking for config at: {config_path}")

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving
from src.features import feat_preprocessing
from src.modeling import modeling

# Передаем путь явно
config = loader.load_config(config_path)
print("✅ Config loaded successfully!")

Looking for config at: D:\Skills\Kaggle\ml-regression_concrete-strength\config\config.yaml
✅ Config loaded successfully!


## 6.1. Загрузка данных

In [10]:
# Загрузка train
df_train = loader.data_load_preprocessed(data_type='train',
                                         config=config)

[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_train.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [11]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


In [15]:
# Загрузка отчета по экспериментам
modeling_result = modeling.load_sorted_modeling_report(config)
modeling_result

,outlier_strategy,model_name,model_type,dataset_size,mean_rmse,std_rmse,training_time_sec,memory_used_mb
0,['abnormal'],CatBoostRegressor,trees_models,770,4.2595,0.3446,13.49,0.44
1,['abnormal'],CatBoostRegressor,linear_models,770,4.2608,0.3868,11.55,12.38
2,"['gost_binar', 'combine']",CatBoostRegressor,linear_models,770,4.3196,0.4173,13.77,0.21
3,"['gost_binar', 'combine']",CatBoostRegressor,trees_models,770,4.3260,0.3906,14.72,0.03
4,['combine'],CatBoostRegressor,linear_models,770,4.3304,0.4210,11.68,0.15
...,...,...,...,...,...,...,...,...
105,['abnormal'],ElasticNet,trees_models,770,8.2662,0.4285,1.57,0.00
106,"['gost_binar', 'combine']",ElasticNet,linear_models,770,8.3014,0.4700,1.47,0.00
107,['gost_binar'],ElasticNet,linear_models,781,8.3183,0.5227,1.38,0.00
108,['abnormal'],ElasticNet,linear_models,770,8.3673,0.4635,1.29,0.00


In [16]:
# Определяем лучшую стратию
best_strategy = modeling.get_best_model_strategy(modeling_result)

best_model = best_strategy['model']
best_model_type = best_strategy['model_type']
best_rmse = best_strategy['rmse']
best_outlier_strategy = eval(best_strategy['outlier_strategy'])